In [12]:
import os

In [13]:
%pwd

'/Users/sairoopesh/Documents/projects/kidney-disease-classification/research'

In [14]:
os.chdir('../')

In [15]:
%pwd

'/Users/sairoopesh/Documents/projects/kidney-disease-classification'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from kidney_disease_classification.constants import *
from kidney_disease_classification.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath =  PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config   

In [19]:
import os
import zipfile
import gdown
from kidney_disease_classification import logger
from kidney_disease_classification.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:

        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion', exist_ok=True)
            logger.info(f"Downloading dataset from {dataset_url} to {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?export=download&id='
            gdown.download(prefix + file_id, str(zip_download_dir))

            logger.info(f"Downloaded dataset from {dataset_url} to {zip_download_dir}")
        
        except Exception as e:
            logger.error(f"Error downloading dataset from {dataset_url} to {zip_download_dir}")
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error("Error in data ingestion")
    raise e

[2024-01-26 19:02:22,424: INFO: common] yaml file: config/config.yaml loaded successfully]
[2024-01-26 19:02:22,425: INFO: common] yaml file: params.yaml loaded successfully]
[2024-01-26 19:02:22,426: INFO: common] created directory at: artifacts]
[2024-01-26 19:02:22,427: INFO: common] created directory at: artifacts/data_ingestion]
[2024-01-26 19:02:22,427: INFO: 3427827848] Downloading dataset from https://drive.google.com/file/d/1CqdxNzwyxcwfnvwHsEwUj_PwnnafHI2e/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1CqdxNzwyxcwfnvwHsEwUj_PwnnafHI2e
From (redirected): https://drive.google.com/uc?export=download&id=1CqdxNzwyxcwfnvwHsEwUj_PwnnafHI2e&confirm=t&uuid=e3e2dd17-fb3f-4746-b882-079a7082b4a6
To: /Users/sairoopesh/Documents/projects/kidney-disease-classification/artifacts/data_ingestion/data.zip
100%|██████████| 1.63G/1.63G [02:37<00:00, 10.3MB/s]

[2024-01-26 19:05:01,801: INFO: 3427827848] Downloaded dataset from https://drive.google.com/file/d/1CqdxNzwyxcwfnvwHsEwUj_PwnnafHI2e/view?usp=sharing to artifacts/data_ingestion/data.zip]
